<a href="https://colab.research.google.com/github/nmuzyka2004/Proyecto-Jupiter_Mercadona/blob/main/Mercadona_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import requests
import pandas as pd
from pandas import json_normalize

In [82]:
# Sacamos categorias y subcategorias
url = "https://tienda.mercadona.es/api/categories/"
response = requests.get(url)
data = response.json()["results"]
# data

In [83]:
# Convertimos JSON de categorias a DataFrame
df_categorias = pd.json_normalize(data, record_path="categories", meta=["id", "name"], record_prefix="subcategory_", meta_prefix="category_")
df_categorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   subcategory_id           151 non-null    int64 
 1   subcategory_name         151 non-null    object
 2   subcategory_order        151 non-null    int64 
 3   subcategory_layout       151 non-null    int64 
 4   subcategory_published    151 non-null    bool  
 5   subcategory_is_extended  151 non-null    bool  
 6   category_id              151 non-null    object
 7   category_name            151 non-null    object
dtypes: bool(2), int64(3), object(3)
memory usage: 7.5+ KB


In [84]:
# Dejamos las columnas necesarias y quitamos otras
df_categorias = df_categorias[["category_id", "category_name", "subcategory_id", "subcategory_name"]]
df_categorias.head()

,category_id,category_name,subcategory_id,subcategory_name
0,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"
1,12,"Aceite, especias y salsas",115,Especias
2,12,"Aceite, especias y salsas",116,"Mayonesa, ketchup y mostaza"
3,12,"Aceite, especias y salsas",117,Otras salsas
4,18,Agua y refrescos,156,Agua


In [85]:
df_categorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   category_id       151 non-null    object
 1   category_name     151 non-null    object
 2   subcategory_id    151 non-null    int64 
 3   subcategory_name  151 non-null    object
dtypes: int64(1), object(3)
memory usage: 4.8+ KB


In [86]:
# Sacamos los datos de los productos
lista_productos = []

for _, row in df_categorias.iterrows():
    categoria_id = row["category_id"]
    categoria_name = row["category_name"]
    subcategoria_id = row["subcategory_id"]
    subcategoria_name = row["subcategory_name"]

    # print(f"Estamos en subcategoria {subcategoria_id}: {subcategoria_name}")

    url = f"https://tienda.mercadona.es/api/categories/{subcategoria_id}"
    response = requests.get(url)
    data = response.json()["categories"]

    df_subcategorias = pd.json_normalize(data, record_path="products", meta=["id", "name"], record_prefix="products_", meta_prefix="subsubcategory_")

    # Dejamos las columnas necesarias y quitamos otras
    df_productos = df_subcategorias.rename(columns={
        "products_display_name": "name",
        "products_packaging": "description",
        "products_price_instructions.unit_size": "unit_size",
        "products_price_instructions.size_format": "size_format",
        "products_price_instructions.unit_price": "price",
        "products_price_instructions.reference_price": "reference_price",
        "products_price_instructions.reference_format": "reference_unit",
        "products_id": "id",
        "subsubcategory_id": "subcategory_2_nivel_id",
        "subsubcategory_name": "subcategory_2_nivel_name"}).copy()

    df_productos = df_productos[["id", "name", "description", "unit_size", "size_format", "price",
                                 "reference_price", "reference_unit", "subcategory_2_nivel_id", "subcategory_2_nivel_name"]]

    # Añadimos nombres y id de categorias y subcategorias
    df_productos["category_id"] = categoria_id
    df_productos["category_name"] = categoria_name
    df_productos["subcategory_id"] = subcategoria_id
    df_productos["subcategory_name"] = subcategoria_name

    # Añadimos df a la lista de productos
    lista_productos.append(df_productos)

# Juntamos todo en un DataFrame
df_productos_mercadona = pd.concat(lista_productos, ignore_index=True)
df_productos_mercadona.head()

,id,name,description,unit_size,size_format,price,reference_price,reference_unit,subcategory_2_nivel_id,subcategory_2_nivel_name,category_id,category_name,subcategory_id,subcategory_name
0,4241,"Aceite de oliva 0,4º Hacendado",Garrafa,5.00,l,19.95,3.990,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"
1,4240,"Aceite de oliva 0,4º Hacendado",Botella,1.00,l,4.45,4.450,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"
2,4717,Aceite de oliva virgen extra Hacendado,Garrafa,3.00,l,15.85,5.284,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"
3,4740,Aceite de oliva virgen extra Hacendado,Botella,1.00,l,5.55,5.550,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"
4,4706,Aceite de oliva virgen extra Hacendado Gran Se...,Botella,0.75,l,6.55,8.734,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"


In [75]:
print(df_productos_mercadona.columns)

Index(['id', 'name', 'description', 'unit_size', 'size_format', 'price',
       'reference_price', 'reference_unit', 'subcategory_2_nivel_id',
       'subcategory_2_nivel_name', 'category_id', 'category_name',
       'subcategory_id', 'subcategory_name'],
      dtype='object')


In [87]:
# ponemos mas información en "discription"
df_productos_mercadona["description"] = (
    df_productos_mercadona["description"].astype(str) + " " +
    df_productos_mercadona["unit_size"].fillna("").astype(str) + " " +
    df_productos_mercadona["size_format"].fillna("").astype(str)
)
df_productos_mercadona.drop(columns=["unit_size", "size_format"], inplace=True)
df_productos_mercadona.head()

,id,name,description,price,reference_price,reference_unit,subcategory_2_nivel_id,subcategory_2_nivel_name,category_id,category_name,subcategory_id,subcategory_name
0,4241,"Aceite de oliva 0,4º Hacendado",Garrafa 5.0 l,19.95,3.990,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"
1,4240,"Aceite de oliva 0,4º Hacendado",Botella 1.0 l,4.45,4.450,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"
2,4717,Aceite de oliva virgen extra Hacendado,Garrafa 3.0 l,15.85,5.284,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"
3,4740,Aceite de oliva virgen extra Hacendado,Botella 1.0 l,5.55,5.550,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"
4,4706,Aceite de oliva virgen extra Hacendado Gran Se...,Botella 0.75 l,6.55,8.734,L,420,Aceite de oliva,12,"Aceite, especias y salsas",112,"Aceite, vinagre y sal"


In [88]:
df_productos_mercadona.to_csv("productos_mercadona.csv", index=False, encoding="utf-8-sig")

In [90]:
df_productos_mercadona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4805 entries, 0 to 4804
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        4805 non-null   object
 1   name                      4805 non-null   object
 2   description               4805 non-null   object
 3   price                     4805 non-null   object
 4   reference_price           4805 non-null   object
 5   reference_unit            4805 non-null   object
 6   subcategory_2_nivel_id    4805 non-null   object
 7   subcategory_2_nivel_name  4805 non-null   object
 8   category_id               4805 non-null   int64 
 9   category_name             4805 non-null   object
 10  subcategory_id            4805 non-null   int64 
 11  subcategory_name          4805 non-null   object
dtypes: int64(2), object(10)
memory usage: 450.6+ KB
